In [1]:
import numpy as np
import math
import random
import pprint
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
class Heat:

    def __init__(self, inputs):
        """
        Generates a grid of size (minT, maxT) x (minX, maxX) with step size of (stepT, stepX).
        :param float minT: minimum value of the interval in t
        :param float maxT: maximum value of the interval in t
        :param float minX: minimum value of the interval in x
        :param float maxX: maximum value of the interval in x
        :param float stepT: step in T dimension
        :param float stepX: step in X dimension
        """
        self.x = None  # x values for grid
        self.t = None  # t values for grid
        self.R = {}  # Perturbation values
        self.u = None
        self.uE = {}  # TODO add exact solution for k = [1/2, 1/2]
        self.h = None
        self.k = None
        self.step_x = inputs['step_x']
        self.step_t = inputs['step_t']
        self.min_t = inputs['min_t']
        self.max_t = inputs['max_t']  # change this to get the desired t
        self.min_x = inputs['min_x']
        self.max_x = inputs['max_x']  # change this to get the desired t

    def f(self, t, k):
        '''
        f is the source function for the heat equation
        :param float t: Single value for t.
        :param list k: Parameters of the function f. Should be a list of 2 values k = [k1,k2]
        '''
        return k[0] * math.exp(-k[1] * t)

    def g(self, x):
        """
        g is the function of u(0,x) = g(x)
        :param x: point x in which we will calculate x
        :return: value of g calculated in point x
        """
        return x*(1-x)

    def equation(self, ut, uxx, f):
        """
        Returns the value of the equation given the parameters
        :param ut: Solution derivate in t. Is a function of parameters (x,t)
        :param uxx: Solution derivate in xx. Is a function of parameters (x,t)
        :param f: Source of heat in the bar. Is a function of parameters (x,t,u). Has parameters (k = [k1,k2])
        """
        return ut - uxx*uxx

    def makeGrid(self):
        """
        Generates a grid of size (minT, maxT) x (minX, maxX) with step size of (stepT, stepX)."""
        self.h = self.step_x
        self.k = self.step_t
        self.t, self.x = np.meshgrid(np.arange(self.min_t, self.max_t, self.step_t), np.arange(self.min_x, self.max_x
                                                                                               , self.step_x))

    def getPerturbation(self, e):
        self.R = {}
        for i in range(0, len(self.t)):
            for j in range(0, len(self.x)):
                self.R[(i,j)] = random.uniform(-e, e)

    def solver(self):
        """
        This solver utilizes a straight-forward Forward Difference Method.
        We followed the tutorial in Burden for the method, but changing the parameters in order to decrease
        computational time.

        :return:
        """
        m = len(self.x)
        n = len(self.t[0])
        h = self.step_x
        k = self.step_t
        print("x: {}".format(self.x.shape))
        self.u = np.zeros(self.x.shape)
        print("u: {}".format(self.u.shape))
        lamb = k/(h*h)
        w = np.zeros(m + 1)
        l = np.zeros(m + 1)
        u = np.zeros(m + 1)
        z = np.zeros(m + 1)
        w[m] = w[0] = 0  # following the initial condition u(0,t) = u(l,t) = 0. If needed, change this.
        for i in range(1,m-1):
            w[i] = self.g(i*h)

        l[1] = 1 + lamb
        u[1] = -lamb/(2*l[1])
        for i in range(2, m-1):
            l[i] = 1 + lamb + lamb*u[i-1]/2
            u[i] = -lamb/(2*l[i])

        l[m-1] = 1 + lamb + lamb*u[m-2]/2
        for j in range(1, n+1):
            t = j*k  # current t
            z[1] = ((1-lamb)*w[1] + lamb/2*w[2] + self.f(t, [1/2, 1/2]))/l[1]
            for i in range(2, m):
                z[i] = ((1 - lamb) * w[i] + lamb / 2 * (w[i+1] + w[i-1] + z[i-1]) + self.f(t, [1/2, 1/2])) / l[i]
            w[m-1] = z[m-1]
            for i in range(m-2, 0, -1):
                w[i] = z[i] - u[i]*w[i+1]

            for i in range(0, m+1):
                x = i*h
                #print(x, w[i])
                #print('oi')
                self.u[i-1,j-1] = w[i]
                self.t[i-1, j-1] = t
                self.x[i-1, j-1] = x

    def plot(self):
        fig = plt.figure()
        ax = plt.axes(projection='3d')

        ax.plot_surface(self.x, self.t, self.u)
        ax.set_xlabel('x')
        ax.set_ylabel('t')
        ax.set_zlabel('u')
        ax.view_init(20, 60)
        fig.show()